Brickinside.com의 특정 레고 모델의 정보 페이지에 접근하여 리뷰를 수집.
수집할 내용은 모델명(csv 파일명에 반영),평점, 닉네임, 짧은 총평.

In [32]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import csv
import re

In [33]:
html = urlopen('https://www.brickinside.com/NeoView.php?Db=set&Mode=view&Number=7525&BackDepth=1')

In [34]:
html

In [35]:
bsobject = bs(html,'html.parser')

In [36]:
bsobject.html

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="brickinside,브릭인사이드,레고,구매,판매,창작,lego,LEGO,brick,moc,커뮤니티,동호회" name="keywords">
<meta charset="utf-8"/>
<script language="javascript" src="skin/script/legoinside/share.js?q=20160414"></script>
<meta content="" name="author"/>
<meta content="~ 2010 : 350,500 원 2011 ~ : 382,000 원.." name="description"/>
<link href="https://www.brickinside.com/NeoView.php?Db=set&amp;Number=7525" rel="canonical"/>
<meta content="1137478036284233" name="fb_app_id" property="fb:app_id">
<meta content="article" property="og:type"/>
<meta content="Tower Bridge (타워 브리지)" property="og:title"/>
<meta content="Brickinside" property="og:site_name"/>
<meta content="https://www.brickinside.com/NeoView.php?Db=set&amp;Number=7525" property="og:url"/>
<meta content="https://www.brickinside.com/sns_image.php?Db=set&amp;Number=7525" property="og:image"/>
<meta con

In [37]:
bsobject.head.title.get_text()

'10214 : Tower Bridge (타워 브리지) - Brickinside'

제품 모델명은 헤드부분 텍스트 맨 앞의 숫자 문자열로 특정 가능. 정규표현식을 활용하여 추출할 수 있음.

In [38]:
opinion = []  
rating = [] 
nickname = [] 

실제 리뷰들의 현황을 확인해 보니, 리뷰만 작성/평점만 부여하는 경우가 있어 find_all 이후 인덱스를 통해 평점과 리뷰를 매치시키기에는 난점이 있음.

따라서 리뷰 내용과 평점, 작성자 닉네임 각각을 위한 공리스트를 만들어 크롤링 한 내용을 append 하고, 만일 해당 내용이 작성되지 않았을 경우에는 'N/A'가 입력될 수 있도록 예외처리 코드를 작성함.

In [39]:
profile_list = bsobject.find_all('td',{'align':'left','width':'10%','valign':'top'})

profile_list 내부에 닉네임, 평점 정보 모두가 존재함. 각각 for 구문을 이용해 리스트 내부로 할당.

In [40]:
for i in profile_list:#평점 처리
    try:
        rating.append(re.search('\w+',i.find('span').get_text()).group())
    except:
        rating.append('N/A')

평점에 해당하는 부분을 find 메소드로 확보 후 정규표현 처리하여 평점에 해당하는 숫자만 가져옴. 만약 평점이 작성되지 않았을 시에는 N/A로 처리.

In [41]:
for i in profile_list:#닉네임 처리
    try:
        nickname.append(i.find('b').get_text())
    except:
        nickname.append('N/A')

b 태그 내부에 있는 사용자 닉네임 정보도 수집하여 리스트에 할당.

In [42]:
opinion_list = bsobject.find_all('tr',{'class':'cmt','onmouseout':"this.style.background=''"})

리뷰가 작성되지 않았을 경우에 대비하여 리뷰가 실제 작성되는 영역보다 한 단계 위의 태그를 find_all로 검색한 후, 평점과 같은 방식으로 처리함.

In [43]:
for i in opinion_list:
    try:
        opinion.append(i.find('td').get_text())
    except:
        opinion.append('N/A')

In [44]:
for i in range(0,len(opinion)): #debugging
    print(str(i+1) + '.' + rating[i] +'\n'+ opinion[i] +'\n'*2)

1.8
오늘 드디어 완성하고 첫 글을 올리는 영광을 가지네요. 크기는 아시는 바와같이 102cm입니다. 그런데 타워의 크기만 본다면 생각보다는 높지가 않더라구요. 그리고 견고함도 조금 떨어지는 감이 있습니다. 그래서 한번 떨어뜨려 재조립했습니다.  버스 택시등 자동차의 디테일은 괜찮네요. 브릭의 수가 4000여개가 넘는데 많은 부분이 작은 타일브릭의 수가 포함되어 생각보다 크기가 커지지 않은 것 같네요. 지루하지 않은 이틀의 작업이었습니다. 만들땐 조심조심... ^^


2.N/A
타워 양쪽이 정확히 같으므로 똑같은 작업의 2번 반복이 지루할 수 있지만 가족과 함께 만든다면 기쁨이 2배가 되는 모델입니다. 개인적으론 딸과 정확히 나눠 손맛을 볼수 있어 기분 좋았던 모델이었습니다. 디테일은 만번대 모델임을 충분히 충족해 주고 있으며 크기 또한 진열해 놓기에  적당한??... 선반을 홀로 차지하며 단품 전시시 매력을 발하는 훌륭한 모델입니다.


3.10
처음 만들어본 초대형 제품입니다. 번호도 써있지 않는 봉지로 인해 보자마자 기겁했지만 어찌어찌 브릭을 분류하고 조립을 했습니다. 저녁 4시쯤 부터 시작해서 완성될때까지 정말 흥미진진 했습니다. 브릭수 2000개 정도의 제품도 다음날을 넘겨 만들정도로 힘든데 신기하게도 이제품은 12시간의 중노동에도 불구하고 시간가는줄 모르고 만들었습니다. 1x1의 슬러프 브릭의 압박이 손가락을 괴롭혔지만 그래도 재미있는 조립, 웅장하고 압도적인 크기, 수려한 외관과 색감.. 무엇하나 부족함이 없는 제품입니다.


4.10
똑같은 작업을 2번 해야한다는 생각에 좀 지루할수도 있겠다 생각했지만 그건 큰 오산이었습니다. 만드는 내내 전혀 지루하지 않았고 오히려 좀 아쉬운 감이 없지 않았습니다.
크기면에선 좀더 컸으면 어땠을까 하는 생각을 해봤습니다. 만번대 건물들과도 나름 잘 어울리고 색감도 이쁘고 전시효과가 아주 좋은 제품이라고 생각합니다..


5.10
추셕연휴동안 만들어 보았습니다..
약 10시간?? 정말 힘들었네요..ㅡㅡ;;

실제로 잘 크롤링이 이루어졌는지 확인하기 위해 화면출력을 시도해 봄.

In [45]:
with open('crawling'+'_model '+re.match('\d+',bsobject.head.title.get_text()).group()+'.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['No','Nickname', 'Rating', 'Opinion'])
        
    for n in range(0, len(opinion)):
        if rating[n]!='N/A':
            rating_in = rating[n]+'점'
        else:
            rating_in = rating[n]
        opinion_in = opinion[n]
        nickname_in = nickname[n] 
        writer.writerow([n+1, nickname_in, rating_in, opinion_in])

크롤링된 내용은 crawling_model ~~ .csv 파일에 저장됨.